In [5]:
import torch


def split_to_float(path):
    """ Read a file, split by line, transfer from string list to float tensor.

    :param path: .txt input path.
    :return: list with (12, n) shape, n = the number of line.
    """
    data2d_file = open(path)
    data2d = []
    for line in data2d_file:
        string_format = line.strip().split(' ')
        float_line = []
        for item in string_format:
            float_line.append(float(item))
        data2d.append(torch.tensor(float_line))
    return data2d


all_2d = split_to_float("./depth_data/data2dtrain.txt")
all_3d = split_to_float("./depth_data/data3d2train.txt")
print(f" - data_2d length: {len(all_2d)}\n"
      f" - 2d sample: {all_2d[0]}\n"
      f" - data_3d length: {len(all_3d)}\n"
      f" - 3d sample: {all_3d[0]}")

train_2d = all_2d[0:7000]
train_3d = all_3d[0:7000]
verify_2d = all_2d[7000:8000]
verify_3d = all_3d[7000:8000]

 - data_2d length: 8000
 - 2d sample: tensor([  0.,   0.,  93., -94., 150.,   5.,   3.,   0.,  81., -94., 154.,   5.])
 - data_3d length: 8000
 - 3d sample: tensor([0.2960, 0.6361, 0.7125, 0.1213, 0.4975, 0.4101])


In [6]:
from torch.utils.data import Dataset


class AngleDataset(Dataset):
    def __init__(self, info_2d, info_3d):
        self.info_2d = info_2d
        self.info_3d = info_3d
        self.length = len(info_2d)

    def __len__(self):
        return self.length

    def __getitem__(self, item):
        if type(item) == int:
            if 0 <= item < self.length:
                return self.info_2d[item], self.info_3d[item]
            else:
                raise IndexError("Index out of range!")
        else:
            raise TypeError("Type of index has to be int!")


train_dataset = AngleDataset(train_2d, train_3d)
print("train dataset size:", len(train_dataset))

verify_dataset = AngleDataset(verify_2d, verify_3d)
print("verify dataset size:", len(train_dataset))

print(train_dataset[0])

train dataset size: 7000
verify dataset size: 7000
(tensor([  0.,   0.,  93., -94., 150.,   5.,   3.,   0.,  81., -94., 154.,   5.]), tensor([0.2960, 0.6361, 0.7125, 0.1213, 0.4975, 0.4101]))


In [7]:
import torch.nn as nn
import torch.nn.functional as f
import torch.optim as optim
from torch.utils.data import DataLoader


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(12, 24)
        self.fc2 = nn.Linear(24, 16)
        self.fc3 = nn.Linear(16, 8)
        self.fc4 = nn.Linear(8, 6)

    def forward(self, x):
        x = f.relu(self.fc1(x))
        x = f.relu(self.fc2(x))
        x = f.relu(self.fc3(x))
        x = self.fc4(x)

        return x


train_loader = DataLoader(train_dataset, batch_size=4)
net = Net()
criterion = nn.MSELoss(reduction="mean")
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


for epoch in range(1):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        item_2d, item_3d = data

        optimizer.zero_grad()
        outputs = net(item_2d)
        loss = criterion(outputs, item_3d)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        if i % 500 == 499:
            print(f"[{epoch + 1}, {i + 1:5d} * 4] loss {running_loss / 500:.3f}")
            running_loss = 0.0

print("Finished Training")

[1,   500 * 4] loss 0.144
[1,  1000 * 4] loss 0.044
[1,  1500 * 4] loss 0.039
Finished Training


In [8]:
verify_loader = DataLoader(verify_dataset, batch_size=4)

running_loss = 0.0
for i, data in enumerate(verify_loader, 0):
    item_2d, item_3d = data
    outputs = net(item_2d)
    loss = criterion(outputs, item_3d)
    running_loss += loss.item()
running_loss /= len(verify_dataset) / 4

print("Verify loss:", running_loss)

Verify loss: 0.03502192367240787


```
Sample verify ground truth:
tensor([[0.5113, 0.1653, 0.6187, 0.5566, 0.0868, 0.5515],
        [0.4644, 0.3139, 0.4590, 0.7452, 0.3005, 0.4782],
        [0.7532, 0.7513, 0.6178, 0.3636, 0.7596, 0.4728],
        [0.6101, 0.4775, 0.6080, 0.5423, 0.7396, 0.7974]])
```
```
Sample verify result:
tensor([[0.5112, 0.3709, 0.4689, 0.5273, 0.4510, 0.5135],
        [0.4717, 0.3608, 0.4494, 0.4892, 0.4455, 0.5010],
        [0.6755, 0.8832, 0.5825, 0.6136, 0.7112, 0.4537],
        [0.4873, 0.3648, 0.4571, 0.5043, 0.4477, 0.5059]])
```